In [1]:
import pandas as pd
import numpy as np
import warnings
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense,Flatten
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from keras.layers import Dropout,Conv1D, MaxPooling1D,UpSampling1D,Reshape,Activation,BatchNormalization,LeakyReLU
from keras.models import Sequential
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
pat = pd.read_csv("./moss_plos_one_data.csv")

In [3]:
true_patient = pat[pat.y==True]
id_true = list(set(list(true_patient.id)))

In [4]:
true_patient = pat[pat.id.isin(id_true)]
true_patient["num"] = 1
true_patient.shape

(189584, 63)

In [5]:
true_patient["cond"] = true_patient["num"].copy()
true_patient.cond[true_patient.y==True] =0

In [6]:
filterPat = true_patient.groupby("id")["cond"].sum().to_frame().reset_index()
filterPat.head()

,id,cond
0,133,159
1,187,660
2,191,1
3,206,239
4,282,610


In [7]:
filtered_id =  list(filterPat.id[filterPat.cond>=24*4])
new_df = true_patient[true_patient.id.isin(filtered_id)]

In [8]:
len(filtered_id)

246

In [9]:
new_df2 = new_df.loc[new_df.y==False]

In [10]:
reverse_df = new_df2[::-1]
reverse_df_24 = reverse_df.groupby("id").head(96)

In [11]:
reverse_df_24.shape

(23616, 64)

In [12]:
reverse_df_12 = reverse_df_24.groupby("id").tail(48)
reverse_df_12.shape

(11808, 64)

In [13]:
df_12 = reverse_df_12[::-1]
df_12.head()

,id,age,race,svc,LOS,ICU_Pt_Days,Mort,n_evts,eid,y,...,s8.edrk,s24.edrk,srr,dfa,cosen,lds,af,AF,num,cond
26387,133,63.7,wht,Other,14,5,0,1,8.0,False,...,-0.000128,-0.000128,0.019054,0.635528,-2.360563,1.915081,0.0,0.0,1,1
26388,133,63.7,wht,Other,14,5,0,1,8.0,False,...,-0.000096,-0.000096,0.030071,0.703619,-2.330572,1.736611,0.0,0.0,1,1
26389,133,63.7,wht,Other,14,5,0,1,8.0,False,...,-0.000071,-0.000071,0.020232,0.587544,-2.325428,1.797390,0.0,0.0,1,1
26390,133,63.7,wht,Other,14,5,0,1,8.0,False,...,-0.000054,-0.000054,0.015003,0.735833,-2.326985,1.729841,0.0,0.0,1,1
26391,133,63.7,wht,Other,14,5,0,1,8.0,False,...,-0.000043,-0.000039,0.028355,0.667666,-2.324768,2.114693,0.0,0.0,1,1


In [14]:
df_12.y = True

In [15]:
id_list = list(set(pat['id']))  # 8105
id_false = [id for id in id_list if id not in id_true]
false_patient = pat[pat.id.isin(id_false)]

In [16]:
false_patient["cond"] = 1
false_count = false_patient.groupby("id")["cond"].sum().to_frame()
false_count = false_count[false_count.cond>48]

In [17]:
false_id = false_count.index

In [18]:
false_patient = pat[pat.id.isin(false_id)]
false_patient.head()

,id,age,race,svc,LOS,ICU_Pt_Days,Mort,n_evts,eid,y,...,edrk,s2.edrk,s8.edrk,s24.edrk,srr,dfa,cosen,lds,af,AF
0,1,39.35,wht,Cardiothoracic Surgery,4,1,0,0,NaN,False,...,23.527702,0.000002,0.000001,-4.918909e-07,0.027926,0.722903,-2.391665,1.670820,0.0,0.0
1,1,39.35,wht,Cardiothoracic Surgery,4,1,0,0,NaN,False,...,25.659513,0.002369,0.002369,2.368679e-03,0.031948,0.728738,-2.432732,1.995264,0.0,0.0
2,1,39.35,wht,Cardiothoracic Surgery,4,1,0,0,NaN,False,...,26.504228,0.001654,0.001654,1.653626e-03,0.023217,0.804216,-2.459413,2.058334,0.0,0.0
3,1,39.35,wht,Cardiothoracic Surgery,4,1,0,0,NaN,False,...,26.203203,0.000986,0.000986,9.856909e-04,0.010812,0.974394,-2.455503,1.986356,0.0,0.0
4,1,39.35,wht,Cardiothoracic Surgery,4,1,0,0,NaN,False,...,24.115443,0.000191,0.000191,1.910191e-04,0.007782,0.768893,-2.429776,1.900642,0.0,0.0


In [19]:
#randomly pick 12 hours 
#split training and test set
false_id = list(false_id)
true_id = list(set(df_12.id))
np.random.seed(200)
np.random.shuffle(list(false_id))
np.random.shuffle(list(true_id))


In [20]:
false_patient["count"] = false_patient.groupby("id").cumcount()

In [21]:
false_patient["count_1"] = false_patient["count"] - 48
# false_patient["count"] = false_patient.groupby("id")["count"].max()

In [22]:
max_num =  false_patient.groupby("id")["count_1"].max()

In [23]:
max_num = max_num.reset_index()
new_false_patient = pd.merge(false_patient,max_num,how="left",on="id")

In [24]:
new_false_patient.drop(["count_1_x"],axis=1,inplace=True)

In [25]:
import random
def agg(x):
    up = random.randint(0,x["count_1_y"].values[1])
    low = up+48
    
    return x.iloc[up:low,]

df3 = new_false_patient.groupby("id").apply(lambda x: agg(x))

In [26]:
df3 = df3.reset_index(drop=True)
df3.drop(["count","count_1_y"],axis=1,inplace=True)

In [27]:
df3.shape

(324864, 62)

In [28]:
df_12.drop(["num","cond"],axis=1,inplace=True)

In [29]:
total = pd.concat([df3,df_12],axis=0)

In [58]:
train = false_id[0:3384] + true_id[0:123]
test = false_id[3384:] + true_id[123:]
all_data = total.drop(['Sgy', 'MET','eid','tte','death','direct','n_evts','LOS', 'ICU_Pt_Days', 'Mort', 'age', 'race', 'svc'], axis=1)
col = ['y',"id",'hr', 's2.hr', 's8.hr', 's24.hr', 'n.edrk',
       'edrk', 's2.edrk', 's8.edrk', 's24.edrk']
all_data = all_data[col]
X_train = all_data[all_data.id.isin(train)]
x_test = all_data[all_data.id.isin(test)]

In [59]:
len(X_train[X_train.y==True])/len(X_train)

0.03507271171941831

In [60]:
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
scaler = StandardScaler()
y_train = X_train.y
X_train = X_train.drop(['y',"id"], axis=1)

X_train = scaler.fit_transform(X_train)
sm = SMOTE(random_state=42)
new_X_train, new_y_train = sm.fit_sample(X_train, y_train)

In [61]:
y_test = x_test['y']
test_id = x_test["id"]
x_test = x_test.drop(['y',"id"], axis=1)
x_test = scaler.fit_transform(x_test)

In [62]:
new_X_train = new_X_train.reshape(new_X_train.shape[0], new_X_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

In [69]:
input_dim = new_X_train.shape[1]
model= Sequential()
model.add(Conv1D(8, 2, input_shape=new_X_train.shape[1:3]))
model.add(BatchNormalization())
model.add(MaxPooling1D(2))
model.add(Conv1D(8, 2))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.01))
model.add(Conv1D(8, 2))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.01))
model.add(MaxPooling1D(2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(6))
model.add(LeakyReLU(alpha=0.01))
model.add(Dense(6))
model.add(LeakyReLU(alpha=0.01))
model.add(Dropout(0.4))
model.add(Dense(1, activation = 'sigmoid'))

In [70]:
nb_epoch = 30
batch_size = 50
model.compile(optimizer='adam', 
                    loss='binary_crossentropy', 
                    metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath="model.h5",
                               verbose=0,
                               save_best_only=True)
model.fit(new_X_train, new_y_train,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(x_test, y_test),
                    verbose=1)

Train on 324864 samples, validate on 168336 samples
Epoch 1/30
324864/324864 [==============================] - 22s 67us/step - loss: 0.6509 - acc: 0.6168 - val_loss: 0.6331 - val_acc: 0.6203
Epoch 2/30
324864/324864 [==============================] - 21s 63us/step - loss: 0.6328 - acc: 0.6474 - val_loss: 0.6171 - val_acc: 0.6379
Epoch 3/30
324864/324864 [==============================] - 21s 64us/step - loss: 0.6275 - acc: 0.6533 - val_loss: 0.5965 - val_acc: 0.6526
Epoch 4/30
324864/324864 [==============================] - 21s 64us/step - loss: 0.6243 - acc: 0.6561 - val_loss: 0.5970 - val_acc: 0.6742
Epoch 5/30
324864/324864 [==============================] - 21s 64us/step - loss: 0.6220 - acc: 0.6575 - val_loss: 0.6104 - val_acc: 0.6510
Epoch 6/30
324864/324864 [==============================] - 21s 64us/step - loss: 0.6202 - acc: 0.6585 - val_loss: 0.6147 - val_acc: 0.6330
Epoch 7/30
324864/324864 [==============================] - 21s 63us/step - loss: 0.6187 - acc: 0.6577 - val

In [71]:
pred = model.predict_proba(x_test)
ind = pred[:,0].argsort()[::-1][:round(len(pred)*0.03)]
test_id = np.array(test_id)
l = list(set(test_id[ind]))
#true patients with event
a = [i for i in l if i in id_true]
len(a)

46

In [72]:
from sklearn.metrics import roc_auc_score
roc_auc_score(np.array(y_test), pred[:,0])

0.6090607440431858

In [73]:
#misclassified non_event patients
len(list(set([i for i in test_id[ind] if i  in false_id])))

775

In [53]:
#total unique patients 
len(list(set(test_id)))

3507